### Ejercicio: Analizar texto

● Utilizar un texto de proyecto Gutenberg en castellano
http://www.gutenberg.org/browse/languages/es

● Contar palabras y ordenar por frecuencia:

        ● Limpiar preludio y licencia de Project Gutenberg
        ● Omitir “palabras vacías” (stop words) y símbolos

● Encontrar personajes

● Hacer un análisis extra a gusto


El Texto a analizar será el libro **Agua de nieve** de Concha Espina 



In [325]:
import textacy
from collections import Counter
from spacy.lang.en.stop_words import STOP_WORDS
from textacy.doc import Doc

In [361]:
#Obtenemos el texto del libro
with open("datasets/agua_de_nieve.txt", "r") as t:
    texto = t.read()

### Contar palabras y ordenar por frecuencia

In [362]:
#Limpiamos preludio y licencia de Project Gutenberg (además tambien decidimos limpiar todo el prologo inicial del libro
#que no aporta a la informacion que estamos buscando extraer)

#removemos la cabecera
texto = texto[2803:]#texto[6984:]
texto[:30]

'LIBRO PRIMERO\n\nLA VIAJERA RUBI'

In [363]:
#removemos toda la informacion al final del libro que no es parte de la historia
texto = texto[:-23612]
texto[-50:]

', como la voz mansa y pura del agua de la nieve...'

Para poder utilizar el modelo de español de spacy es necesario descargarlo. Para esto ejecutar en la consola la siguiente linea:  

**spacy install es_core_news_sm**

In [364]:
# Cargamos el modelo para el idioma español y parseamos el texto para obtener un "bag-of-terms" en doc
nlp = textacy.load_spacy('es_core_news_sm', disable=('parser',))
proc_text = nlp(texto)
type(proc_text)

spacy.tokens.doc.Doc

In [365]:
proc_text[:100]

LIBRO PRIMERO

LA VIAJERA RUBIA




I

EN EL LAZARETO DE SAN SIMÓN.--REGINA DE ALCÁNTARA.--EL ESPEJO
TURBIO.--LOS MISTERIOS DE UNA NOCHE DE MAYO.--LA FELICIDAD DESCALZA.


TOCÓ el bote dulcemente en la tierra, tierra frondosa y húmeda que
emergía de las aguas como un jirón de los blandos vergeles submarinos.
Regina de Alcántara, moza elegante y gentilísima, de ojos negros y
cabellos rubios, desembarcó de un salto, rápida y leve, sin advertir que
un pasajero le tendía

In [366]:
stop = list(STOP_WORDS)
def clean_doc(doc):
    #Limpiamos puntuacion, simbolos y stopwords del texto y convertimos los token a minuscula
    print("Cleaning")
    # limpiamos puntuacion simbolos y stopwords
    doc = [tok.text for tok in doc if (tok.text not in stop and tok.pos_ != "PUNCT" and tok.pos_ != "SYM")]
    # convertimos a minuscula
    doc = [tok.lower() for tok in doc]
    doc = ' '.join(doc)
    print("Doc cleaned")
    return nlp(doc)

In [367]:
clean_text = clean_doc(proc_text) # type spacy.tokens.doc.Doc
doc = Doc(clean_text) # type textacy.doc.Doc --> for using the method 'to_bag_of_words'


Cleaning
Doc cleaned


In [368]:
bow_doc = doc.to_bag_of_words(as_strings=True);

In [369]:
#Palabras ordenadas por frecuencia de mayor a menor
words_counts = Counter(bow_doc)
sorted(words_counts.items(), key=lambda x: x[1], reverse=True)

[('y', 3333),
 ('á', 1417),
 ('comer', 429),
 ('regina', 413),
 ('parir', 330),
 ('ojo', 170),
 ('entrar', 168),
 ('sobrar', 164),
 ('querer', 161),
 ('mujer', 148),
 ('amor', 147),
 ('mozo', 141),
 ('corazón', 139),
 ('niño', 134),
 ('mirar', 132),
 ('madre', 129),
 ('don', 116),
 ('sentir', 115),
 ('vida', 110),
 ('voz', 109),
 ('casar', 104),
 ('triste', 103),
 ('mar', 100),
 ('muchacho', 96),
 ('bajar', 95),
 ('parecer', 94),
 ('do', 93),
 ('carlos', 90),
 ('carlota', 89),
 ('hijo', 89),
 ('joven', 83),
 ('llegar', 82),
 ('dama', 82),
 ('maría', 80),
 ('señor', 79),
 ('eugenia', 78),
 ('manuel', 78),
 ('adolfo', 77),
 ('ó', 77),
 ('dolor', 77),
 ('ana', 77),
 ('alma', 74),
 ('dulce', 73),
 ('luz', 73),
 ('buscar', 73),
 ('mundo', 73),
 ('ramírez', 72),
 ('velasquín', 72),
 ('volver', 71),
 ('alcántara', 70),
 ('preguntar', 69),
 ('padre', 68),
 ('coser', 67),
 ('sombrar', 66),
 ('cielo', 66),
 ('flor', 65),
 ('dejar', 64),
 ('fué', 64),
 ('pasar', 62),
 ('tierra', 61),
 ('asir', 61

### Encontrar personajes

In [370]:
# tipos de entidades presentes en el texto
set(ent.label_ for ent in proc_text.ents)

{'LOC', 'MISC', 'ORG', 'PER'}

In [371]:

characters = Counter()
# De la documentación de spacy una "named entity" es cualquier objeto del mundo real como una persona,
# ubicación, organización o producto con un nombre propio. Utilizaremos las 'named entities' para encontrar los nombres 
# de personas mas comunes en el texto, suponiendo que estos serán los personajes principales.


for nam_ent in proc_text.ents:
    if nam_ent.label_ == 'PER':
        characters[nam_ent.lemma_] += 1

characters.most_common()

[('Regina', 104),
 ('Carlota', 74),
 ('Carlos', 63),
 ('Ana María', 56),
 ('Adolfo', 54),
 ('Manuel', 52),
 ('Pero', 38),
 ('Velasquín', 37),
 ('Daniel', 33),
 ('Jaime', 29),
 ('Marta', 28),
 ('Y', 28),
 ('Pablo', 26),
 ('Velasco', 23),
 ('', 22),
 ('De', 18),
 ('don Celso', 17),
 ('Así', 15),
 ('El', 13),
 ('La', 12),
 ('Al', 11),
 ('Robledo _', 11),
 ('Mercedes', 10),
 ('Amador', 9),
 ('Fué', 9),
 ('No', 9),
 ('Estrada', 8),
 ('Ella', 8),
 ('Carlos Ramírez', 8),
 ('Ibarrola', 8),
 ('Eugenia', 7),
 ('Allí', 7),
 ('Después', 6),
 ('Mas', 6),
 ('Timonel _', 6),
 ('Jaime de Alcántara', 6),
 ('Dios', 6),
 ('¡ Qué', 6),
 ('Juan Ramírez', 6),
 ('Manuel Velasco', 6),
 ('Aquella', 5),
 ('Pero Regina', 5),
 ('Aquel', 5),
 ('Ya', 4),
 ('Va', 4),
 ('Por', 4),
 ('Las', 4),
 ('Rolando', 4),
 ('En', 4),
 ('Carlitos Ramírez', 4),
 ('Ha', 4),
 ('Con', 4),
 ('don Juan', 4),
 ('Había', 4),
 ('Alonso', 4),
 ('Gabriel _', 4),
 ('Marín', 3),
 ('Fabricio', 3),
 ('Carlota de Heredia', 3),
 ('Mi', 3),
 ('Aún

Los modelos utilizados para taggear las 'named entities' son modelos estadisticos y la clasificación que realizan
dependen de los ejemplos con los que fueron entrenados. Esto puede conducir a errores como los que vemos en la lista
anterior.
 
También podemos ver que hay palabras que seguramente se estan refiriendo a una persona y probablemente el modelo
piense que es una persona por el contexto en el cual esta siendo utilizada.
 
Si tenemos que hacer una primer predicción de quienes son los personajes principales diriamos que, por diferencia de  frecuencia:
 
Los principales son: Regina, Carlota y Carlos
Seguidos de Ana María, Adolgo y Manuel.

### Intentaremos buscar si existe algún otro personaje principal que los modelos hayan pasado por alto 

In [372]:
places = Counter()

for nam_ent in proc_text.ents:
    if nam_ent.label_ == 'LOC':
        places[nam_ent.lemma_] += 1

places.most_common()

[('', 887),
 ('Regina', 141),
 ('Alcántara', 41),
 ('Torremar', 32),
 ('Eugenia', 24),
 ('Ramírez', 22),
 ('¡', 14),
 ('Dolores', 9),
 ('Alcoy', 8),
 ('Madrid', 7),
 ('Carlitos', 6),
 ('Qué', 6),
 ('Después', 5),
 ('¡ Qué', 4),
 ('Bernaldo', 4),
 ('Europa', 4),
 ('Spa', 4),
 ('Heredia', 4),
 ('Mercedes', 4),
 ('Sí', 4),
 ('Andes', 4),
 ('¡ Si', 4),
 ('VII', 3),
 ('Francia', 3),
 ('Están', 3),
 ('Pilcomayo', 3),
 ('A', 3),
 ('Amaba', 3),
 ('Intrusa', 3),
 ('Rhin', 3),
 ('Tlaloc', 3),
 ('La', 3),
 ('Arfeo', 2),
 ('Selva Negra', 2),
 ('Sintióse', 2),
 ('San Martín', 2),
 ('IV', 2),
 ('Plata', 2),
 ('América', 2),
 ('Santander', 2),
 ('Plaza Mayor', 2),
 ('Pues', 2),
 ('Quedó', 2),
 ('Parecía', 2),
 ('Van', 2),
 ('Fermín', 2),
 ('Cómo', 2),
 ('Vigo', 2),
 ('Miraba', 2),
 ('Argentina', 2),
 ('Allí', 2),
 ('Cuba', 2),
 ('Tengo', 2),
 ('Silvia', 2),
 ('Florencia', 2),
 ('Ibarrola', 2),
 ('Roma', 2),
 ('Torpemente', 2),
 ('No', 2),
 ('Casino', 2),
 ('Gran Chaco', 2),
 ('Bebe', 2),
 ('Busca', 2

Aquí aparece el personaje de Regina nuevamente, que creemos es el personaje principal de la historia. Una posibilidad
por la cual haya sido clasificada como un lugar es que durante el libro es nombrada en reiteradas ocasiones como 
Regina de Alcántara. Siendo Alcántara un lugar (municipio de españa)

### Dado que Regina y Carlota parecen ser los dos personajes principales, ¿En cuantas oraciones aparecen juntas? ¿En cuantas aparecen cada una por separado?

In [386]:
reg_car_sent = [sent for sent in proc_text.sents if ('Regina' in sent.string and 'Carlota' in sent.string)]
reg_sent = [sent for sent in proc_text.sents if 'Regina' in sent.string]
car_sent = [sent for sent in proc_text.sents if 'Carlota' in sent.string]
total_sent = [sent for sent in proc_text.sents]

print("Regina y Carlota aparecen juntas en {0} de {1} oraciones".format(len(reg_car_sent), len(total_sent)))
print("Regina aparece en {0} de {1} oraciones".format(len(reg_sent), len(total_sent)))
print("Carlota aparece en {0} de {1} oraciones".format(len(car_sent), len(total_sent)))


Regina y Carlota aparecen juntas en 14 de 2764 oraciones
Regina aparece en 405 de 2764 oraciones
Carlota aparece en 96 de 2764 oraciones


In [387]:
# Observando el texto se puede ver que por algún motivo la versión de proyecto gutemberg se encuentra mal escrita con 
# lo cual se dificultando mucho la tarea de los modelos y por ende de todos los analisis posibles.
# por ejemplo: Veamos las ultimas 3 oraciones donde regina y carlota aparecen juntas

reg_car_sent[-3:]


[La sublimidad
 de Carlota, liberta de su cruel esclavitud por el amor, y esclava, por
 el amor mismo, en un convento, resultábale á Regina tan misteriosa y
 vaga como el impulso de «la novia de Rolando», cautiva de sagrada
 clausura por creer á su amante víctima de la guerra., El
 matrimonio, en apariencia semejante al que la dama rubia conmemora en
 cruel aniversario, sabe ella que es en el fondo muy distinto del suyo, y
 admira con humildad sus nobles fundamentos: quiere Carlota partir con su
 hijo para consagrarse á levantar el vuelo de aquel mustio corazón, en
 saludable mudanza de horizontes; pero antes paga sus deudas de gratitud
 á la ilustre familia de Velasco y ofrece á su hija la ventura, con
 suprema generosidad, empujándola hacia el palacio del valle, donde
 siembre sonrisas y consolaciones sobre la memoria de Velasquín;
 consiente Manuel, devoto cumplidor de sus palabras y preclaro artífice
 de bellas obras, y doña Mercedes abre sus brazos temblorosos para
 abrazarse al «

**Aquí podemos ver que se hace abuso del recurso '...' y que la puntuación en general no se encuentra bien utilizada.**